# Removing Unfair Bias in Machine Learning

AI can embed human and societal bias and be then deployed at scale. Many algorithms are now being reexamined due to illegal bias. So how do you remove bias & discrimination in the machine learning pipeline? In this workshop you will learn the debiasing techniques that can be implemented by using the open source toolkit [AI Fairness 360](https://github.com/IBM/AIF360). 
 
**AI Fairness 360** (AIF360) is an extensible, open source toolkit for measuring, understanding, and removing AI bias. It contains the most widely used bias metrics, bias mitigation algorithms, and metric explainers from the top AI fairness researchers across industry & academia. 

In this notebook you will: 
* apply a practical use case of bias measurement & mitigation
* measure bias in data & models 
* apply the fairness algorithms to reduce bias

## 1. Import packages

### Install aif360

After running the below cell, restart the kernel before continuing with the rest of the notebook.

In [ ]:
!pip install lime
!pip install aif360

In [ ]:
import sys
sys.path.insert(1, "../")  

# data exploration
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

np.random.seed(0)

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import lime
import lime.lime_tabular

# aif360 data, metrics and algorithms
from aif360.datasets import GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.datasets.lime_encoder import LimeEncoder 

from IPython.display import Markdown, display
%matplotlib inline

## 2. Explore the data

### Load data

In [ ]:
aif360_location = !python -c "from distutils.sysconfig import get_python_lib; print(get_python_lib())"
import os
install_loc = os.path.join(aif360_location[0], "aif360/data/raw/german/")
%cd $install_loc

In [ ]:
!wget ftp://ftp.ics.uci.edu/pub/machine-learning-databases/statlog/german/german.data
!wget ftp://ftp.ics.uci.edu/pub/machine-learning-databases/statlog/german/german.doc
%cd -

In [ ]:
dataset_german = GermanDataset()

### AIF360 data format

All variables of this dataset are described in the [documentation](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.GermanDataset.html) with more details in the description of the [`StandardDataset`](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.datasets.StandardDataset.html). In short, the dataset class contains a numpy array or pandas DataFrame with several variables. 

In [ ]:
type(dataset_german)

In [ ]:
type(dataset_german.features)

In [ ]:
print(f'labels: {dataset_german.label_names}')
print(f'protected attributes: {dataset_german.protected_attribute_names}')
print(f'number of features: {len(dataset_german.feature_names)}')

### Explore with pandas

<div class="alert alert-info" style="font-size:100%">
<b>If you are new to Pandas read this <a href="http://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html">10 minute introduction</a> for a quick overview.<br>
</div>

Convert the data to a `features` DataFrame and `labels` Series:

In [ ]:
features = pd.DataFrame(dataset_german.features, columns=dataset_german.feature_names)
labels = pd.Series(dataset_german.labels.ravel(), name=dataset_german.label_names[0])

In [ ]:
features.describe()

### Explore the distribution of the features

In [ ]:
plt.rcParams["figure.figsize"] = (18,18)

features.hist();
plt.tight_layout()

Most features are binary, but a few are continuous:

In [ ]:
features[['credit_amount','month','number_of_credits']]. \
        plot(subplots=True, \
             kind='hist', \
             layout=(2, 2),
             sharex=False, \
             figsize=(10, 10));

And also add the labels:

In [ ]:
[fig, axs] = plt.subplots(2, 2, figsize=(14,8))
fig.add_subplot(2, 2, 1)
features['credit_amount'].plot(kind='hist',bins=12);
fig.add_subplot(2, 2, 2)
features['month'].plot(kind='hist',bins=12);
fig.add_subplot(2, 2, 3)
features['number_of_credits'].plot(kind='hist',bins=12);
fig.add_subplot(2, 2, 4)
labels.plot(kind='hist',bins=12);
plt.tight_layout()

### Explore bias in the data

Bias could occur based on age or sex in this dataset. 

* set the protected attribute to be `age`, `age >=25` is considered privileged
* this dataset also contains protected attribute for `sex` that are not consider in this evaluation
* split the original dataset into training and testing datasets
* set two variables for the privileged (1) and unprivileged (0) values for the age attribute. These are key inputs for detecting and mitigating bias

<div class="alert alert-success">
 <b>OPTIONAL EXERCISE</b> <br/> 
 To explore the gender bias in the data, edit the below code to use `sex` as the protected attribute and assign new privileged and unprivileged groups.
</div>

In [ ]:
dataset_german = GermanDataset(protected_attribute_names=['age'],
                    privileged_classes=[lambda x: x >= 25],      
                    features_to_drop=['personal_status', 'sex']) 

dataset_german_train, dataset_german_test = dataset_german.split([0.7], shuffle=True)

privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

### Metrics based on a single `BinaryLabelDataset`

<div class="alert alert-info" style="font-size:100%">
<b>Read <a href="https://aif360.readthedocs.io/en/latest/modules/generated/aif360.metrics.BinaryLabelDatasetMetric.html">the documentation</a> for a full overview of this class and a list of all bias metrics. <a href="http://aif360.mybluemix.net/data">This demo</a> contains definitions of the metrics as well.<br>
</div>

In [ ]:
metric_german_train = BinaryLabelDatasetMetric(dataset_german_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

metric_german_test = BinaryLabelDatasetMetric(dataset_german_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

In [ ]:
help(metric_german_train)

### Exploring bias metrics
* `mean_difference`: alias of `statistical_parity_difference` 
    * Difference of the rate of favorable outcomes received by the unprivileged group to the privileged group. 
    * A negative value indicates less favorable outcomes for the unprivileged groups
    * The ideal value of this metric is 0
    * Fairness for this metric is between -0.1 and 0.1

* `disparate_impact`: ratio of rate of favorable outcome for the unprivileged group to that of the privileged group

In [ ]:
display(Markdown("#### Original training dataset"))
print("mean_difference = %f" % metric_german_train.mean_difference())
print("disparate_impact = %f" % metric_german_train.disparate_impact())
print("consistency = %f" % metric_german_train.consistency())
print("base_rate = %f" % metric_german_train.base_rate())
print("num_negatives = %f" % metric_german_train.num_negatives())
print("num_positives = %f" % metric_german_train.num_positives())
print("smoothed_empirical_differential_fairness = %f" % metric_german_train.smoothed_empirical_differential_fairness())

## 3. Select and transform features to build a model

### Classification problem

From the above
- mostly binary data, a few with a few classes and two continuous. 
- label is binary: 0 or 1 is what will be predicted, so this will be a **binary classification**

Some of the model options:
- logistical regression
- Decision trees
- Random forests
- Bayesian networks
- Support vector machines
- Neural networks
- Logistic regression

### Scale and normalise features

- one-hot encoding for multiple classes
- features need to be standardised, from same distribution
- no missing values
- ...

[StandardScaler](https://scikit-learn.org/stable/modules/preprocessing.html) - 
*Standardization of datasets is a common requirement for many machine learning estimators implemented in scikit-learn; they might behave badly if the individual features do not more or less look like standard normally distributed data: Gaussian with zero mean and unit variance. `StandardScaler` implements the Transformer API to compute the mean and standard deviation on a training set so as to be able to later reapply the same transformation on the testing set.*

aif360 format can be used with scikitlearn!

In [ ]:
# scale data
scale_german = StandardScaler().fit(dataset_german_train.features)

X_train = scale_german.transform(dataset_german_train.features)
y_train = dataset_german_train.labels.ravel()
w_train = dataset_german_train.instance_weights.ravel()

X_test = scale_german.transform(dataset_german_test.features)
y_test = dataset_german_test.labels.ravel()
w_test = dataset_german_test.instance_weights.ravel()

In [ ]:
# what does the data look like now?
plt.rcParams["figure.figsize"] = (18,18)

scaled_features = pd.DataFrame(X_train, columns=dataset_german.feature_names)

scaled_features.hist();
plt.tight_layout()

In [ ]:
scale_german.mean_

In [ ]:
scale_german.scale_

## 4. Build models

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

### Train on the original data

In [ ]:
# Logistic regression classifier and predictions

# create an instance of the model
lmod = LogisticRegression()

# train the model
lmod.fit(X_train, y_train, 
         sample_weight=dataset_german_train.instance_weights)

# calculate predicted labels
y_train_pred = lmod.predict(X_train)

# assign positive class index
pos_ind = np.where(lmod.classes_ == dataset_german_train.favorable_label)[0][0]

# add predicted labels to predictions dataset
dataset_german_train_pred = dataset_german_train.copy()
dataset_german_train_pred.labels = y_train_pred

In [ ]:
# model accuracy
score = lmod.score(X_test, y_test)
print(score)

In [ ]:
# confusion matrix
cm = metrics.confusion_matrix(y_test, lmod.predict(X_test))

plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');

### Remove bias by reweighing data

**Reweighing** is a preprocessing technique that weights the examples in each (group, label) combination differently to ensure fairness before classification.

<div class="alert alert-info" style="font-size:100%">
<b>Read <a href="https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.preprocessing.Reweighing.html">the documentation</a> for a full overview<br>
</div>

In [ ]:
RW = Reweighing(unprivileged_groups=unprivileged_groups,
               privileged_groups=privileged_groups)

# compute the weights for reweighing the dataset
RW.fit(dataset_german_train)

# transform the dataset to a new dataset based on the estimated transformation
dataset_transf_train = RW.transform(dataset_german_train)
dataset_transf_test = RW.transform(dataset_german_test)

In [ ]:
display(Markdown("#### Original training dataset"))
print("mean_difference = %f" % metric_german_train.mean_difference())
print("disparate_impact = %f" % metric_german_train.disparate_impact())
print("consistency = %f" % metric_german_train.consistency())
print("base_rate = %f" % metric_german_train.base_rate())
print("num_negatives = %f" % metric_german_train.num_negatives())
print("num_positives = %f" % metric_german_train.num_positives())
print("smoothed_empirical_differential_fairness = %f" % metric_german_train.smoothed_empirical_differential_fairness())

metric_transf_train = BinaryLabelDatasetMetric(dataset_transf_train, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)

display(Markdown("#### Reweighted training dataset"))
print("mean_difference = %f" % metric_transf_train.mean_difference())
print("disparate_impact = %f" % metric_transf_train.disparate_impact())
print("consistency = %f" % metric_transf_train.consistency())
print("base_rate = %f" % metric_transf_train.base_rate())
print("num_negatives = %f" % metric_transf_train.num_negatives())
print("num_positives = %f" % metric_transf_train.num_positives())
print("smoothed_empirical_differential_fairness = %f" % metric_german_train.smoothed_empirical_differential_fairness())

#### Train on reweighted data

In [ ]:
# scale data
scale_transf = StandardScaler().fit(dataset_transf_train.features)

X_train_transf = scale_transf.transform(dataset_transf_train.features)
y_train_transf = dataset_transf_train.labels.ravel()
w_train_transf = dataset_transf_train.instance_weights.ravel()

X_test_transf = scale_transf.transform(dataset_transf_test.features)
y_test_transf = dataset_transf_test.labels.ravel()
w_test_transf = dataset_transf_test.instance_weights.ravel()

In [ ]:
# create a new instance of the model
lmod_transf = LogisticRegression()

# train the model
lmod_transf.fit(X_train_transf, y_train_transf, 
         sample_weight=dataset_transf_train.instance_weights)

# calculate predicted labels
y_train_pred_transf = lmod_transf.predict(X_train_transf)

# assign positive class index
pos_ind_transf = np.where(lmod_transf.classes_ == dataset_transf_train.favorable_label)[0][0]

# add predicted labels to predictions dataset
dataset_transf_train_pred = dataset_transf_train.copy()
dataset_transf_train_pred.labels = y_train_pred_transf

In [ ]:
# model accuracy
print(score)
score2 = lmod_transf.score(X_test_transf, y_test_transf)
print(score2)

In [ ]:
# confusion matrix
cm_transf = metrics.confusion_matrix(y_test_transf, lmod_transf.predict(X_test_transf))

plt.figure(figsize=(5,5))
sns.heatmap(cm_transf, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
plt.title('Reweighted model');

In [ ]:
# confusion matrix
cm = metrics.confusion_matrix(y_test, lmod.predict(X_test))

plt.figure(figsize=(5,5))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
plt.title('Original model');

# age = 1: is > 25s privileged 

## 5. Evaluate models

Exploring [LIME](https://lime-ml.readthedocs.io/en/latest/)

In [ ]:
limeData = LimeEncoder().fit(dataset_german_train)
s_train = limeData.transform(dataset_german_train.features)
s_test = limeData.transform(dataset_german_test.features)

scale = scale_german

model = lmod    

explainer = lime.lime_tabular.LimeTabularExplainer(s_train ,class_names=limeData.s_class_names, 
                                                   feature_names = limeData.s_feature_names,
                                                   categorical_features=limeData.s_categorical_features, 
                                                   categorical_names=limeData.s_categorical_names, 
                                                   kernel_width=3, verbose=False,discretize_continuous=True)

s_predict_fn = lambda x: model.predict_proba(scale.transform(limeData.inverse_transform(x)))

display(Markdown("#### Original training dataset"))

i1 = 1
exp = explainer.explain_instance(s_test[i1], s_predict_fn, num_features=5)
exp.show_in_notebook(show_all=False)
print("        Actual label: " + str(dataset_german_test.labels[i1]))

i2 = 100
exp = explainer.explain_instance(s_test[i2], s_predict_fn, num_features=5)
exp.show_in_notebook(show_all=False)
print("        Actual label: " + str(dataset_german_test.labels[i2]))


In [ ]:
limeData = LimeEncoder().fit(dataset_german_train)
s_train = limeData.transform(dataset_german_train.features)
s_test = limeData.transform(dataset_german_test.features)

scale = scale_transf
model = lmod_transf   

s_predict_fn = lambda x: model.predict_proba(scale.transform(limeData.inverse_transform(x)))

display(Markdown("#### Reweighted training dataset"))

i1 = 1
exp = explainer.explain_instance(s_test[i1], s_predict_fn, num_features=5)
exp.show_in_notebook(show_all=False)
print("        Actual label: " + str(dataset_german_test.labels[i1]))

i2 = 100
exp = explainer.explain_instance(s_test[i2], s_predict_fn, num_features=5)
exp.show_in_notebook(show_all=False)
print("        Actual label: " + str(dataset_german_test.labels[i2]))


### Author
Margriet Groenendijk is a Data & AI Developer Advocate for IBM. She develops and presents talks and workshops about data science and AI. She is active in the local developer communities through attending, presenting and organising meetups and conferences. She has a background in climate science where she explored large observational datasets of carbon uptake by forests during her PhD, and global scale weather and climate models as a postdoctoral fellow.

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.